In [1]:
import numpy as np
import pandas as pd
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import os
import datetime
from collections import Counter
import pickle
import time
import timeit
import math
from sklearn import preprocessing
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from scipy.spatial import distance
import re
from itertools import groupby
from operator import itemgetter
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn.externals.six import StringIO  
from IPython.display import Image  
# import pydot

%matplotlib inline
%timeit
# %load_ext cython

In [2]:
## Get data
# path = os.getcwd()
path = '/Users/yingjiang/Dropbox/Learnings/Stats_data/Projects/FBCheckins'
os.listdir(path)
location = pd.read_csv(path + '/train.csv')

## Subset data by accuracy
locAcc = location[(location.accuracy > 70) & (location.accuracy < 400)]

## Create time features
locTime = locAcc
locTime['timeFmtted'] = pd.to_datetime(locTime.time, unit = 'm')
locTime['hour'] = locTime.timeFmtted.apply(lambda x: datetime.datetime.strptime(str(x)[11:], "%H:%M:%S").hour)
locTime['weekday'] = locTime.timeFmtted.apply(lambda x: x.dayofweek)
locTime['month'] = locTime.timeFmtted.apply(lambda x: x.month)
locTime['isPeak'] = np.where(((locTime['hour'] >= 6) & (locTime['hour'] <= 9)) | ((locTime['hour'] >= 17) & (locTime['hour'] <= 21)), 1, 0)
locTime['isNight'] = np.where((locTime['hour'] >= 0) & (locTime['hour'] <= 4), 1, 0)
locTime['isWeekend'] = np.where((locTime['weekday']==5) | (locTime['weekday']==6), 1, 0)
locTime['isHoliday'] = np.where((locTime['month'] == 7) | (locTime['month'] == 8) | (locTime['month'] == 12) | (locTime['month'] == 8), 1, 0)

/Users/yingjiang/miniconda2/lib/python2.7/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/yingjiang/miniconda2/lib/python2.7/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/yingjiang/miniconda2/lib/python2.7/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/

In [3]:
## Make tr and cv sets. Try a small subset first.
locSmall = locTime.iloc[:100000]
locSmall = locSmall.drop(['row_id', 'accuracy', 'time', 'timeFmtted'], axis = 1)

X = locSmall.drop('place_id', axis = 1)
Y = locSmall.place_id

X_train, X_ts, Y_train, Y_ts = train_test_split(X, Y, test_size=0.20, train_size=0.80)
X_tr, X_cv, Y_tr, Y_cv = train_test_split(X_train, Y_train, test_size = 0.2, train_size = 0.8)

In [6]:
## Decision tree classifier

# Decision criterion = 'Gini'
clf1_16K = tree.DecisionTreeClassifier()
clf1_16K = clf1_16K.fit(X_tr, Y_tr)
pred1_16K = clf1_16K.predict(X_cv) 
sum(pred1_16K == Y_cv) / float(len(Y_cv)) # cv size = 16K; accuracy = 15.1%

0.15106249999999999

In [ ]:
clf1_32K = tree.DecisionTreeClassifier()
clf1_32K = clf1_32K.fit(X_tr, Y_tr)
pred1_32K = clf1_32K.predict(X_cv) 
sum(pred1_32K == Y_cv) / float(len(Y_cv)) # cv size = 16K; accuracy = 14.4%

In [ ]:
# Decision criterion = 'entropy'
clf2_16K = tree.DecisionTreeClassifier(criterion = 'entropy')
clf2_16K = clf2_16K.fit(X_tr, Y_tr)
pred2_16K = clf2_16K.predict(X_cv)
sum(pred2_16K == Y_cv) # cv size = 1600; accuracy = 30/1600 = 2%

In [ ]:
# Tuning:
# 1. Decision criterion: Gini or entropy didn't make a difference.
# 2. min_samples_split. When increased, accuracy decreased.
# 3. Increase in sample size (1.6K to 8K). Accuracy increased from 2% to 8%.

In [ ]:
## Random forest classifier

# Decision criterion = 'Gini'
clf3 = RandomForestClassifier(n_estimators=10)
clf3 = clf3.fit(X_tr, Y_tr)
pred3 = clf3.predict(X_cv)

In [14]:
sum(pred3 == Y_cv)

7

In [31]:
## Visualize decision trees
# dot_data = StringIO()
# tree.export_graphviz(clf, out_file=dot_data)
tree.export_graphviz(clf, out_file='tree1.dot')
# graph = pydot.graph_from_dot_data(dot_data.getvalue())  
# Image(graph.create_png())  

In [32]:
tree.export_graphviz(clf, out_file='tree1.dot') #produces dot file

# dotfile = StringIO()
# tree.export_graphviz(clf, out_file=dotfile)
# pydot.graph_from_dot_data(dotfile.getvalue()).write_png("tree1.png")

ImportError: No module named pydot

In [18]:
def visualize_tree(tree, feature_names):
    """Create tree png using graphviz.

    Args
    ----
    tree -- scikit-learn DecsisionTree.
    feature_names -- list of feature names.
    """
    with open("tree1_16K.dot", 'w') as f:
        tree.export_graphviz(tree, out_file=f,
                        feature_names=feature_names)

    command = ["dot", "-Tpng", "dt.dot", "-o", "dt.png"]
    try:
        subprocess.check_call(command)
    except:
        exit("Could not run dot, ie graphviz, to "
             "produce visualization")

In [19]:
visualize_tree(clf, X_tr.columns.tolist())

AttributeError: 'DecisionTreeClassifier' object has no attribute 'export_graphviz'

In [9]:
X_tr.columns.tolist()

['x',
 'y',
 'hour',
 'weekday',
 'month',
 'isPeak',
 'isNight',
 'isWeekend',
 'isHoliday']